In [36]:
from google.colab import files

from bs4 import BeautifulSoup
import csv
import requests#, xlsxwriter
import pandas as pd
import datetime

def get_batch_group(batch):
    if batch in ['01','05','09','13']:
        return 'Mateen'
    elif batch in ['02','06','10','14']:
        return 'Faryad'
    elif batch in ['03','07','11','15']:
        return 'Liaqat'
    elif batch in ['04','08','12','16']:
        return 'Shah Nawaz'

current_date = datetime.date.today()

payload = {'UN': 'amo11216', 'PW': 'lesco123'}
url     = 'http://116.71.135.214:36247/MIS/checklogin.asp'

s = requests.Session()
r = s.post(url, data=payload)
try:
    r.raise_for_status()
except requests.exceptions.HTTPError:
    print('oops bad status code {} on request!'.format(r.status_code))
else:
    print('our login redirected to: {}'.format(r.url))

url   = 'http://116.71.135.214:36247/MIS/RECOVERY_BATCH_WISE_REV2.asp?offcode=11216&off=SUB%20DIVISION:11216%20(LIAQAT%20ABAD)'
html  = s.get(url).text
soup  = BeautifulSoup(html)
table = soup.select_one("body > table:nth-child(2)")

header = []
rows = []
for i, row in enumerate(table.find_all('tr')):
    if i in [0,1,2]:# i==0:
        header = [el.text.strip() for el in row.find_all('th')]
    else:
        rows.append([el.text.strip() for el in row.find_all('td')])

headers = ['BATCH','DUE DATE','COMP. BILLING','LPS LVD','PVT BILLING WITH LPS','GVT BILLING WITH LPS','TOTAL BILLING WITH LPS',
'CUR ASMNT',	'%AGE'	,'ARREAR'	,'%AGE',	'P.DISC',	'%AGE'	,'PVT PMNT',	'GVT PMNT',	'TOTAL'	,'%AGE',
'GOVT CONS',	'GOVT AMNT',	'PVT CONS',	'PVT AMNT',	'P. DISC CONS'	,'P. DISC AMNT',	'TOTAL CONS',	'TOTAL AMNT']
df = pd.DataFrame(rows, columns = headers)

df['LINEMAN']         = df['BATCH'].apply(get_batch_group)
df['TARGET RUNNING']  = df['PVT AMNT'].apply(pd.to_numeric) * 0.1
df['TARGET DEAD']     = df['P. DISC AMNT'].apply(pd.to_numeric) * 0.1

df.to_excel(str(current_date) + '.xlsx')
files.download(str(current_date) + '.xlsx')


our login redirected to: http://116.71.135.214:36247/MIS/Menu.asp


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>